# 痞客邦資料分析

## Preparation

In [56]:
# module import
import re
import json
from pprint import pprint

In [2]:
# the file path of pixnet
data_file_path = "./2017-pixnet-hackathon-data/article/food.json"

## Load the json file

In [57]:
data = []
with open(data_file_path) as data_file:
    for line in data_file:
        data.append(json.loads(line))

In [139]:
print "Number of blogs: {}".format(len(data))

Number of blogs: 9635


In [ ]:
# filter out data without content
data = [d for d in data if 'content' in d]


## Parse the phone

In [137]:
#phone_pattern = r"電話?([0-9\-])"
phone_pattern = r"電話.*?([0-9\-\(\)]+)"
debug_pattern = "電話.{50}"

In [138]:
# pattern refine process
for d in data:
    if 'content' not in d:
        continue
    if not re.findall(phone_pattern, d['content'].encode('utf-8')):
        if re.findall(debug_pattern, d['content'].encode('utf-8')):
            for bug in re.findall(debug_pattern, d['content'].encode('utf-8')):
                print bug

電話．．．全是只有小時候回爺爺家才會�
電話：（０２）２９１０－９９５８</span><
電話：０９２８９７８１９９（Gary）</stron
電話跟老闆問清楚，老闆說成立的理想是�
電話預訂或臉書Messenger留言預訂，最晚為�
電話後，得知鳳山有這樣的素食大街，聽�
電話進來，其他用餐或準備時段店家都是�
電話訂位的難度也不低，想預約得多點耐�
電話通知，但店家還限制了每日登記人數�
電話去很順利地就完成了訂位，容易的程�
電話，不過我還是有體會到探索廚房電話�
電話確認看看這才發現還真不太好打...好�
電話都沒人接，最後只好親自跑去潮肉訂�
電話臨時預約晚上的用餐事宜，接通時的�
電話那頭突然講了一堆聽不懂的語言，到�
電話那端傳來親切的聲音，我毫不猶豫的�
電話預約，依照貴賓訂位日期以及人數安�
電話那頭，威宏告訴我，現在台南高鐵與�
電話預約，所以有心想吃到，並不算難，</
電話，先去別處晃晃，店家會電話通知有�
電話訂位，結果訂了兩次都沒有訂到，第�
電話號碼，好奇地問老闆娘藍記究竟有啥�
電話預定，省下來的時間能更快享用熱呼�
電話訂位注文小隔間，才能享受早午餐的�
電話預約外帶也是不錯的選擇！</span></span>
電話預約外帶，營業時間更改如下</span></em
電話訂位，免得真的擠不進好擠！</span></sp
電話訂位時確認可以拍照，坐定後就先在�
電話預約</strong></strong></span></strong></span></p
電話預約</strong></strong></strong></strong></span><
電話預約</strong></strong></strong></strong></strong
電話、技術或其他不可歸責於本格之事由�
電話：（０２）２２１９－０９８０</span><
電話詢問一下<br />（以上價格都需要另外�
電話來時問我中午吃啥?我都說我中午吃飼�
電話亭奶茶！在高雄也創下大排長龍的紀�
電話亭，非常跳痛的大膽設計，讓人想一�
電話預約喔 &nbsp; 或者先來這訂位再去附近
電話預約<br />採用FB訊息訂位，月初就要開
電話號碼，師傅就會盡量避開前次的菜色�
電話訂位時，告知當天用餐是特別節日慶�
電話預約了九點開始營業的時間，</span></p>
電話

In [134]:
print ['０１２３４５６７８９']

['\xef\xbc\x90\xef\xbc\x91\xef\xbc\x92\xef\xbc\x93\xef\xbc\x94\xef\xbc\x95\xef\xbc\x96\xef\xbc\x97\xef\xbc\x98\xef\xbc\x99']


In [127]:
print re.findall(phone_pattern, data[100]['content'].encode('utf-8'))

電話


In [64]:
re.findall(phone_pattern, json.dumps(data[100]['content'], ensure_ascii=False))

[]

In [67]:
print json.dumps(data[100]['content'], ensure_ascii=False)

"<p><span style=\"font-family: 'comic sans ms', sans-serif; font-size: 14pt; color: #000000;\">♘<span style=\"color: #ff6600;\">♖</span>♘<span style=\"color: #ff6600;\">♖</span>♘<span style=\"color: #ff6600;\">♖</span><strong>Tame Moose</strong>♘<span style=\"color: #ff6600;\">♖</span>♘<span style=\"color: #ff6600;\">♖</span>♘<span style=\"color: #ff6600;\">♖</span></span><br /><img title=\"cats.jpg\" src=\"http://pic.pimg.tw/joyaijia/1428597064-1813900297_n.jpg\" alt=\"cats.jpg\" border=\"0\" /> &nbsp;&nbsp;&nbsp;</p>\r\n<p><span style=\"color: #000000;\">♕ 雙手奉上店家資訊 ♛</span><br /><span style=\"color: #000000;\">地址：台北市大同區建泰里南京西路18巷6弄6號1樓</span><br /><span style=\"color: #000000;\">交通方式：從捷運中山站一號出口，牛乳大王巷口走進去，第二條巷子就看得到囉</span><br /><span style=\"color: #000000;\">營業時間：週二～週日11：-00～23：00</span><br /><span style=\"color: #000000;\">店家電話：2556-5908</span><br /><span style=\"color: #000000;\">有無收取服務費：Yes</span><br /><span style=\"color: #000000;\">有無提供Wifi：Yes</span><br /><span style=\"color: #

In [ ]:
for d in data:
    d['content']

In [61]:
print(data[0]['content'].encode('utf-8'))

<div class="piliapp-mobile-move-screen" style="width:296px;height:43px !important;border:dashed 2px black;font-size:20px !important;line-height:43px !important;background-color:lightblue !important;padding:0 !important;text-align:center;margin-bottom:400px;margin-top:-10px;margin-left:auto;margin-right:auto;">我搬家了！請點<a href="http://jennifer4.com/2014-01-27-390/" style="font-size:28px !important;" title="[捷運美食-新埔站] 板橋人氣店~巴克斯美式小館，不用去吃到飽也能吃巧又吃到很飽!→→恭喜本文獲選PCHOME精選文章和熱門圖片">這裡繼續</a><br><span style="font-size:12px !important;">(工具邦 技術提供)</span></div>
<div class="piliapp-mobile-move-screen" style="width:100%;height:43px !important;border-top:dashed 2px black;border-bottom:dashed 2px black;font-size:20px !important;line-height:43px !important;background-color:lightblue !important;padding:0 !important;text-align:center; position:fixed !important; top:0px !important; left:0 !important;z-index:10000 !important;">我搬家了！請點<a href="http://jennifer4.com/2014-01-27-390/" style="font-size:28px !important

In [11]:
print(data[0]['content']).decode('unicode_escape')

UnicodeEncodeError: 'ascii' codec can't encode characters in position 311-317: ordinal not in range(128)

In [21]:
print(json.dumps(data[100], ensure_ascii=False)) 

{"category": "美味食記", "content": "<p><span style=\"font-family: 'comic sans ms', sans-serif; font-size: 14pt; color: #000000;\">♘<span style=\"color: #ff6600;\">♖</span>♘<span style=\"color: #ff6600;\">♖</span>♘<span style=\"color: #ff6600;\">♖</span><strong>Tame Moose</strong>♘<span style=\"color: #ff6600;\">♖</span>♘<span style=\"color: #ff6600;\">♖</span>♘<span style=\"color: #ff6600;\">♖</span></span><br /><img title=\"cats.jpg\" src=\"http://pic.pimg.tw/joyaijia/1428597064-1813900297_n.jpg\" alt=\"cats.jpg\" border=\"0\" /> &nbsp;&nbsp;&nbsp;</p>\r\n<p><span style=\"color: #000000;\">♕ 雙手奉上店家資訊 ♛</span><br /><span style=\"color: #000000;\">地址：台北市大同區建泰里南京西路18巷6弄6號1樓</span><br /><span style=\"color: #000000;\">交通方式：從捷運中山站一號出口，牛乳大王巷口走進去，第二條巷子就看得到囉</span><br /><span style=\"color: #000000;\">營業時間：週二～週日11：-00～23：00</span><br /><span style=\"color: #000000;\">店家電話：2556-5908</span><br /><span style=\"color: #000000;\">有無收取服務費：Yes</span><br /><span style=\"color: #000000;\">有無提供Wifi：Yes</s